In [1]:
pip install tqdm joblib

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd #data manipulation
from tqdm.auto import tqdm #progress bar
from rdkit import Chem #Chemistry
from rdkit.Chem import Descriptors
import numpy as np #matrix algebra
#from lightgbm import LGBMRegressor, plot_importance #LightGBM models
from sklearn.model_selection import train_test_split #ML trainining
from sklearn.metrics import r2_score, mean_squared_error #ML stats
import seaborn as sns #Plotting
#from yellowbrick.regressor import prediction_error, ResidualsPlot

In [2]:
dfTrain = pd.read_csv("training_data.csv")
dfTest = pd.read_csv("test_data.csv")

In [3]:
smiles_stringsTrain = dfTrain['SMILES'].tolist()
smiles_stringsTest = dfTest['SMILES'].tolist()

In [4]:
smilesTrain = [Chem.MolFromSmiles(mol) for mol in smiles_stringsTrain]
smilesTest = [Chem.MolFromSmiles(mol) for mol in smiles_stringsTest]

In [5]:
descrsTrain = [Descriptors.CalcMolDescriptors(mol) for mol in smilesTrain]
descrsTest = [Descriptors.CalcMolDescriptors(mol) for mol in smilesTest]
df2Train = pd.DataFrame(descrsTrain)
df2Test = pd.DataFrame(descrsTest)

In [6]:
train_X = df2Train.bfill()
train_y = dfTrain.pIC50
test_X = df2Test.bfill()
test_y = dfTest.pIC50

In [7]:
from tqdm import tqdm
import joblib
import numpy as np

from joblib import Parallel, delayed
from tqdm.auto import tqdm

class TQDMParallel(Parallel):
    def __init__(self, *args, **kwargs):
        self._pbar = kwargs.pop('pbar', None)
        super().__init__(*args, **kwargs)

    def __call__(self, *args, **kwargs):
        with tqdm(total=len(self._iterable), desc=self._pbar) as self._pbar:
            return super().__call__(*args, **kwargs)

    def print_progress(self):
        if self._pbar is not None:
            self._pbar.n = self.n_completed_tasks
            self._pbar.refresh()
        else:
            super().print_progress()

In [8]:
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [12]:
# Create the RandomForest model
regr = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)

# Patch joblib's Parallel to use the custom TQDMParallel
joblib.Parallel = TQDMParallel

# Train the model
with joblib.parallel_backend('threading', n_jobs=-1):
    regr.fit(train_X, train_y)
    y_pred = regr.predict(test_X)

In [14]:
# Coefficient of determination
r2 = r2_score(test_y, y_pred)
# Root mean squared error
rmsd = mean_squared_error(test_y, y_pred)**0.5
# Bias
bias = np.mean(y_pred - test_y)
# Standard deviation of the error of prediction
sdep = np.mean(((y_pred - test_y) - np.mean(y_pred - test_y))**2)**0.5

print(r2)
print(rmsd)
print(bias)
print(sdep)
plt.plot(y_pred, test_y)
plt.show()

0.0007012504979255763
1.1717760751143211
0.1560502958152952
1.1613386566313388


NameError: name 'plt' is not defined

In [ ]:
# Initial parameters for RF hyperparameter tuning:

hyper_cv = 10

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 300, num = 10)]

# Number of features to consider at every split
#max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Collect all hyperparameter values:
init_param_grid = {'n_estimators': n_estimators,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}

# Set up hyperparameter tuning using a random grid search over
# different combinations of hyperparameters:
rf = RandomizedSearchCV(estimator = RandomForestRegressor(),
                        param_distributions = init_param_grid,
                        n_iter = 10,
                        cv = hyper_cv,
                        refit = True,
                        verbose = 1,
                        n_jobs = 1)

# Train RF model:
rf.fit(train_X, train_y)

# Use trained RF model to predict y data for the test set:
y_pred = rf.predict(text_X)

# Assess performace of model based on predictions:

# Coefficient of determination
r2 = r2_score(test_y, y_pred)
# Root mean squared error
rmsd = mean_squared_error(test_y, y_pred)**0.5
# Bias
bias = np.mean(y_pred - test_y)
# Standard deviation of the error of prediction
sdep = np.mean(((y_pred - test_y) - np.mean(y_pred - test_y))**2)**0.5

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomFore

In [ ]:
print(r2)
print(rmsd)
print(bias)
print(sdep)